In [1]:
import tweepy
import json
import pandas as pd
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
from config import consumer_key, consumer_secret, access_token, access_token_secret

In [3]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [4]:
tweets = api.user_timeline(screen_name='itsJeffTiedrich', 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False, 
                           tweet_mode = 'extended'
                           )

for info in tweets[:2]:
    #print( info )
    print(f"\n {info['full_text']}")

{'created_at': 'Mon Mar 20 20:57:36 +0000 2023', 'id': 1637921352488304641, 'id_str': '1637921352488304641', 'full_text': '@TristanSnell "trump force one" *rolls eyes* \nhttps://t.co/ur14DMn5oR', 'truncated': False, 'display_text_range': [14, 69], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'TristanSnell', 'name': 'Tristan Snell', 'id': 15801906, 'id_str': '15801906', 'indices': [0, 13]}], 'urls': [{'url': 'https://t.co/ur14DMn5oR', 'expanded_url': 'https://twitter.com/itsJeffTiedrich/status/1501957164142931971', 'display_url': 'twitter.com/itsJeffTiedric…', 'indices': [46, 69]}]}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': 1637916079904137217, 'in_reply_to_status_id_str': '1637916079904137217', 'in_reply_to_user_id': 15801906, 'in_reply_to_user_id_str': '15801906', 'in_reply_to_screen_name': 'TristanSnell', 'user': {'id': 1009577803304656896, 'id_str': '1009577803304656896', 'name': 'J

In [22]:
search_results = api.search(q="BITCOIN", count=100)
for r in search_results['statuses']:
    print (r['text'])
    print ('------------')

Ethereum has taken what was a four-function calculator of a programming language in Bitcoin and turned it into a fu… https://t.co/FqRAMn9xLf
------------
RT @DocumentingBTC: Africa now has the largest amount of #Bitcoin peer-to-peer trading volume in the world—even more than North America. 🌍…
------------
RT @BitMartExchange: 📢 El Salvador installs 200 Bitcoin ATMs - allowing its citizens to convert the cryptocurrency into USD and withdraw it…
------------
RT @MartyBent: The fact that the central bankers are clueless about how #Bitcoin is already helping millions of people around the world is…
------------
RT @oytun86: Madem uzun bir aradan sonra Bitcoin 50 bin doların üzerine çıktı ben de bir çekiliş yapayım. 5 kişiye toplamda 5.000 TL’lik #B…
------------
RT @adnradiochile: 🎙 EN VIVO | En #TuNuevoADN conversamos con @leomeyer sobre lo que deben saber los emprendedores acerca de las criptomone…
------------
RT @MMCrypto: THE US-DOLLAR WILL CEASE TO EXIST, #BITCOIN HOWEVER IS FOR ETERN

In [15]:
public_tweets = api.user_timeline('pattonoswalt', count=25)
for d in public_tweets:
    print('---------------')
    print(d['text'])

---------------
New @harpers with essays by Garret Keizer, Ann Patchett, @harikunzru PLUS an article by @Bernstein?!?!? Whooo! (*Th… https://t.co/C1JDX4BQ9h
---------------
😂😂😂😂😂😂 https://t.co/tsHZZBrJg3
---------------
RT @SimuLiu: “86% on RottenTomatoes? What happened to the other 14?”

-my parents, probably
---------------
hate that little asshole https://t.co/ExYRQCH7lB
---------------
RT @TheOnion: Teary-Eyed Wrestlers Bid Farewell To Friends Made At SummerSlam https://t.co/xf26qV3kgo https://t.co/RPfbSChpgd
---------------
RT @patrickcotnoir: The new @GLucasTalkShow episode with guests @PFTompkins and @richsommer is very, very fun...it featured unexpected call…
---------------
😂😂😂😂😂😂😂 https://t.co/6uJpbP7Fsy
---------------
RT @dantelfer: Be suspicious of liberal vaccine talk. They are in the pocket of Big Alive and want you to not die so they can make money of…
---------------
This is SO beyond gratifying, St. Louis! Second show added at @ThePageantSTL!!!! See you then? https://

In [ ]:
# User Accounts
news_orgs = ['BBC', 'CNN', 'CBSNews', 'FoxNews', 'nytimes']

In [ ]:
#create dataframe with new columns
df_tweets = pd.DataFrame()
df_tweets['ORG'] = ''
df_tweets['TWEET_TEXT'] = ''
df_tweets['TWEET_DATE'] = ''

#make api call. run loop to save dataframe to local, get data from local only
#put the breaks on my loop to keep from too many calls.  Originaly set to 10,000
max_records = -1

i = 0

for org in news_orgs:
    if i > max_records:
        break
    public_tweets = api.user_timeline(org, count=100)
    for tweet in public_tweets:
        df_tweets.loc[i, 'ORG'] = org
        df_tweets.loc[i, 'TWEET_TEXT'] = tweet['text']
        df_tweets.loc[i, 'TWEET_DATE'] = tweet['created_at']
        i = i + 1
    
if len(df_tweets) > 1:
    df_tweets.to_csv('df_tweets.csv', index=False)
        
df_tweets=pd.read_csv('df_tweets.csv')
df_tweets

In [ ]:
#convert dates
df_tweets['TWEET_DATE']=pd.to_datetime(df_tweets['TWEET_DATE']).dt.date
df_tweets.loc[df_tweets['TWEET_DATE'] == pd.to_datetime('2020-11-15')]

In [ ]:
analyser = SentimentIntensityAnalyzer()
print(analyser.polarity_scores('LOVE'))
print(analyser.polarity_scores('EVIL'))

In [ ]:
#create plot
df_filtered = df_tweets

for i, row in df_tweets.iterrows():
    dic = analyser.polarity_scores(row['TWEET_TEXT'])
    df_tweets.loc[i, 'Negative'] = dic['neg']
    df_tweets.loc[i, 'Neutral'] = dic['neu']
    df_tweets.loc[i, 'Positive'] = dic['pos']
    df_tweets.loc[i, 'Compound'] = dic['compound']

df_filtered = df_filtered[df_filtered['TWEET_DATE'] == pd.to_datetime('2020-11-15')]
for org in news_orgs:
    df=df_filtered[df_filtered['ORG']==org]
    df=df.sort_values(by='TWEET_DATE', inplace=False)
    plt.scatter( range(len(df)), df['Compound'])
#format plot    
plt.style.use('ggplot')   
plt.title('Sentiment Analysis of Media Tweets November 15, 2020')
plt.ylabel('Tweet Polarity')
plt.xlabel('Tweets Ago')

#export .png of figure
plt.savefig("Fig1.png")
plt.show()

In [ ]:
for org in news_orgs:
    df=df_filtered[df_filtered['ORG']==org]
    df=df.sort_values(by='TWEET_DATE', inplace=False)
    plt.bar(df['ORG'], df['Compound'])
#format plot    
plt.style.use('ggplot')   
plt.title('Overall Media Stentiment Based on Twitter November 15, 2020')
plt.ylabel('Tweet Polarity')
plt.xlabel('Tweets Ago')

#export .png of figure
plt.savefig("Fig2.png")
plt.show()